# Motivation

The opening of society during an active Covid19 pandemic means reproductive rate $R_0$ is a function of time $R_0 \rightarrow R_0(t)$. In an effort to gain insight to possible reopening methods and consequences of demographic stochasticity. As such a model is needed with the ability to paramaterize the spread rate to an observable. It is common to use the confirmed cases as a metric, but historical under reporting during other epidemics makes it unceartain. Instead we will attempt to relate other data such as the number of deaths and the hospitalization data to parameterize $R_0$.

## Basic SEIR Model

Current concensus among medical research suggest that the incubation time for Covid19 has a typical range of $\tau_inc = \in [2,14]$ days(CITE). As such we will include the exposed population function. All in all, susceptable $S(t)$, exposed $E(t)$, infectious $I(t)$, removed $R(t)$ persons are going to be tracked. The following assumptions are made regarding changing populations. Any change in *susceptible* population is a result of birth/deaths or exposure to the infection. People catagorized as *exposed* either die of natural causes or enter an *infectious* period. Those that become infectious either recover or die as a result of the virus or by natural cuases. Finally, the *removed* population can only decrease from natural causes.The simplest equations for rates of change in S, E, I, R are below. 

$$\frac{dS}{dt} = \mu N - S \left( \beta \frac{I}{N} + \mu \right) $$

$$\frac{dE}{dt} = \beta \frac{S I}{N} - E(\sigma + \mu) $$

$$\frac{dI}{dt} = \sigma E - I(\gamma + \mu) $$

$$\frac{dR}{dt} = \gamma I - \mu R$$

### Function Arguements

Above the rates moving people from one category to another are taken to be constant.Eventually, we will add a driving function which varies over time. A list of the variable and thier meaning follows:

- $\mu$ = *Births/Deaths frequency* $[T^{-1}]$
- $\beta$ = *Contact frequency* $[T^{-1}]$
- $\gamma$ = *Removal frequency* $[T^{-1}]$
- $\sigma$ = *Latent frequncy* $[T^{-1}]$
- $\gamma$ = *Removal frequency* $[T^{-1}]$

The only known variable is $\mu$, found using US cencus population data. The basic reproduction number, $R_0$, relates the contact and  .The other variables must be derived by paramaterizing the rates in terms of reported observables. Additionally, demographic effects can be modelled by introducing a contact driving function. Counter spread measures can be implimented by breaking the pandemic into different timme periods with varying reproduction values or functions.






The set of ODE's can be easily solved using scipy's odeint(). The first requirement is to make a function which supplies the system of equations. 

In [4]:
import pylab as py
from scipy.integrate import odeint

def diff_eqs(Init, t, args):
    mu, beta, = args # arguements
    S, I, E, R = Init # Initial conditions
    dSdt = mu - S * (beta * I + mu) # Change in susceptible population
    dEdt = beta * I * S - E * (sigma + mu) # Change in Exposed population
    dIdt = sigma * E - I * (gamma + mu) # change in Infected population
    dRdt = gamma * I - mu * R # Change in removed individuals
    return dSdt, dIdt, dRdt